# 8. Кейс-стади

In [ ]:
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

### Задание 1
Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?

2) Критикам нравятся больше игры на PC или на PS4?

3) Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [ ]:
games = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/stat_case_study/vgsales.csv')
games

1) Как критики относятся к спортивным играм?

- H0: средняя удовлетворенность критиков играми в жанре Sports >= 70 баллов
- H1: средняя удовлетворенность критиков играми в жанре Sports < 70 баллов

Пусть alpha = 0.05

In [ ]:
sport = games[['Name', 'Genre', 'Global_Sales', 'Critic_Score']]
sport = sport[sport['Genre'] == 'Sports']

sport.describe()

In [ ]:
sport[sport['Critic_Score'].isnull()]

**Давайте посмотрим на гистограмму**

In [ ]:
plt.hist(sport['Critic_Score'])

In [ ]:
alpha = 0.05
result = st.ttest_1samp(sport['Critic_Score'], 70, alternative='less')

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу, среднее меньше 70 баллов')
else:
    print('Не отвергаем нулевую гипотезу')

Не отвергаем нулевую гипотезу


- опишите полученные результаты статистического теста:

Критики (при том, что в данных около 50% игр не имеют оценки) достаточно высоко оценивают игры в жанре Sport, средняя оценка превышает 70 баллов из 100 возможных.

2) Критикам нравятся больше игры на PC или на PS4?

- H0: удовлетворенность критиков играми на PC & PS4 одинаковая
- H1: удовлетворенность критиков играми на PC & PS4 разная (больше или меньше, там посмотрим)

Пусть alpha = 0.05

In [ ]:
platform = games[['Name', 'Platform', 'Global_Sales', 'Critic_Score']]
platform = platform[(platform['Platform'] == 'PC') | (platform['Platform'] == 'PS4')]

platform.info()

In [ ]:
plt.hist(platform['Critic_Score'])

In [ ]:
pl_PC = platform[platform['Platform'] == 'PC']
pl_PS4 = platform[platform['Platform'] == 'PS4']

In [ ]:
alpha = 0.05
result = st.ttest_ind(pl_PC['Critic_Score'], pl_PS4['Critic_Score'], equal_var=False)

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу, удовлетворенность критиков играми на PC & PS4 разная')
else:
    print('Не отвергаем нулевую гипотезу')

Вывод: так как мы не отвергаем H0, то можем сказать, что игроки в целом одинаково удовлетворены играми на PC и играми на PS4.
Войну между пэкашниками и консольщиками можно считать завершённой :) 

<small>Хотя в шутеры с геймпада рубиться - это для извращенцев))))) такое вот имхо</small>

3) Критикам больше нравятся стрелялки или стратегии?

- H0: критики одинаково оценивают стрелялки и стратегии
- H1: критики по-разному оценивают стрелялки и стратегии

Пусть alpha = 0.05

In [ ]:
games['Genre'].unique()

In [ ]:
huinya = games[['Name', 'Genre', 'Global_Sales', 'Critic_Score']]
shoot = huinya[huinya['Genre'] == 'Shooter']
strat = huinya[huinya['Genre'] == 'Strategy']

shoot.info(), strat.info()

In [ ]:
alpha = 0.05
result = st.ttest_ind(shoot['Critic_Score'], strat['Critic_Score'], equal_var=False)

if result.pvalue < alpha:
    print('Отвергаем нулевую гипотезу, критики по-разному оценивают стрелялки и стратегии')
else:
    print('Не отвергаем нулевую гипотезу')

Не отвергаем нулевую гипотезу


результат странный, надо посмотреть визуально на распределение оценок

In [ ]:
sns.boxplot(shoot['Critic_Score'])
plt.show()
sns.boxplot(strat['Critic_Score'])

хм, действительно оценки критиков шутеров и стратегий не сильно различаются, окей

### Задание 2
Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

- Привидите весь текст к нижнему регистру;
- Удалите мусорные символы;
- Удалите стоп-слова;
- Привидите все слова к начальной форме;
- Преобразуйте все сообщения в вектора TF-IDF. Можете поэкспериментировать с параметрами TfidfVectorizer;
- Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;
- Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
- Опишите результаты при помощи confusion_matrix;
- Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [28]:
spam = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/stat_case_study/spam.csv')
spam.loc[spam.Category == 'ham', 'Class'] = 1
spam.loc[spam.Category == 'spam', 'Class'] = 0

spam['Class'] = spam['Class'].astype('int')
spam = spam.drop(['Category'], axis=1)

In [29]:
spam_class = spam['Class']

In [30]:
#Привидите весь текст к нижнему регистру
spam['Message'] = spam['Message'].str.lower()

#Удалите мусорные символы
spam['Message'] = spam['Message'].replace(['[\W_]+'], [' '], regex=True)

In [ ]:
#Удалите стоп-слова:
spam['Message'] = spam['Message'].str[1:-1].str.split(' ').tolist()

nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
spam['Message'] = pd.Series([[word for word in text if word not in stopwords_set] for text in spam['Message']])

In [ ]:
#Привидите все слова к начальной форме:
nltk.download('omw-1.4')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

spam['Message'] = pd.Series([wordnet_lemmatizer.lemmatize(word) for word in text] for text in spam['Message'])

In [33]:
#Превращаем в строку, иначе fit_transform заругается
spam['Message'] = spam['Message'].apply(lambda x: ' '.join(x))

In [34]:
#Преобразуйте все сообщения в вектора TF-IDF:

tfidf = TfidfVectorizer(stop_words=stopwords_set)
tfidf_matrix = tfidf.fit_transform(spam['Message'])
names = tfidf.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

In [ ]:
tfidf_matrix.describe()

In [37]:
#Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42

x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, spam_class, random_state = 42, test_size = 0.30)

In [ ]:
#Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных

lr = LogisticRegression(random_state=42).fit(x_train, y_train)
y_pred = lr.predict(x_test)
lr.predict_proba(x_test)

lr.score(x_test, y_test)

In [ ]:
#Опишите результаты при помощи confusion_matrix

confusion_matrix(y_test, pd.Series(y_pred))

In [ ]:
#Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно 
# (с указанием фактического и предсказанного)

predict = lr.predict(tfidf_matrix)
final_df = pd.DataFrame()
final_df['Message'] = spam['Message']
final_df['fact'] = spam['Category']
final_df['pred'] = predict

final_df

In [ ]:
df_final = final_df[(final_df['fact'] != final_df['pred'])]